# Import libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets
!pip install unsloth
!pip install unbabel-comet
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
base_path = "/content/drive/MyDrive/historical_mt"


Mounted at /content/drive
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

# Import libraries

In [2]:
import os
import pandas as pd
import json
from comet import download_model, load_from_checkpoint

# Set Paths and Parameters

In [13]:
# Base path
#base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# Model Parameters
unsloth_model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'
company_name = 'meta'
model_name = unsloth_model_name.split('/')[1]

In [14]:
# Translation Directions
translation_directions = ['DE_to_EN', 'EN_to_DE']

# List of shot values
shots_list = [2**i for i in range(8)]
shots_list.insert(0, 0)

# List of formatted shot values
formatted_shots_list = [f"{shot:03d}" for shot in shots_list]
formatted_shots_list = formatted_shots_list + ['finetuning']

# Median splitting?
median_splitting = True

# Results path
results_dir = os.path.join(base_path, 'results', company_name, model_name)

# Print paths
print('Base path:', base_path)
print('Results path:', results_dir)
print('Translation directions:', translation_directions)
print('Shots list:', shots_list)
print('Formatted shots list:', formatted_shots_list)
print(f'Median splitting: {median_splitting}')

Base path: /content/drive/MyDrive/historical_mt
Results path: /content/drive/MyDrive/historical_mt/results/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit
Translation directions: ['DE_to_EN', 'EN_to_DE']
Shots list: [0, 1, 2, 4, 8, 16, 32, 64, 128]
Formatted shots list: ['000', '001', '002', '004', '008', '016', '032', '064', '128', 'finetuning']
Median splitting: True


# Function to load JSON file

In [15]:
# Function to load json file
def load_file(file_path):
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

# Function to run inference with COMET model

In [16]:
def run_comet(source_language, target_language, mt_column, current_df, model):

    # Transform current DataFrame into the desired list of dictionaries for the COMET model
    current_data = current_df.apply(lambda row: {'src': row[source_language], 'mt': row[mt_column], 'ref': row[target_language]}, axis=1).tolist()

    # Predict COMET scores
    model_output = model.predict(current_data, batch_size=8, gpus=1)

    # Return inferences
    return model_output

# Function for median splitting

In [17]:
def split_by_median(source_language, median_splitting, current_df):

    if median_splitting:

        # Compute word count based on the source language
        source_language_word_count = f'{source_language}_word_count'
        current_df[source_language_word_count] = current_df[source_language].apply(lambda x: len(str(x).split()))

        # Compute median word count in source language
        median_word_count = current_df[source_language_word_count].median()

        # Create short and long dataframe
        current_df_short = current_df[current_df[source_language_word_count] <= median_word_count]
        current_df_long = current_df[current_df[source_language_word_count] > median_word_count]

        # Print the shapes of datasets
        print("Shape of current_df_short:", current_df_short.shape)
        print("Shape of current_df_long:", current_df_long.shape)
        print(f'Median word count in {source_language}: {median_word_count}')

        # Return median_splitted dataframes
        return ['current_df_short', 'current_df_long'], [current_df_short, current_df_long]

    return ['Full Dataset'], [current_df]

# Load COMET model

In [18]:
# Load model
model_path = download_model("Unbabel/wmt22-comet-da")

# Load the model checkpoint:
model = load_from_checkpoint(model_path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


# Run inference with COMET model

In [19]:
for translation_direction in translation_directions:

    # Define source and target language based on translation_direction
    if translation_direction == 'DE_to_EN':
        source_language = "Early Modern Bohemian German"
        target_language = "English"
    else:
        source_language = "English"
        target_language = "Early Modern Bohemian German"

    print('______________________________________________________________')
    print('______________________________________________________________')
    print(translation_direction)
    print('______________________________________________________________')
    print('______________________________________________________________')

    # For a given translation_direction go through all shots + finetuning
    for shot in formatted_shots_list:

        if shot == 'finetuning':
            file_path = os.path.join(results_dir, 'finetuning', translation_direction, f'{translation_direction}_finetuning.json')
        else:
            file_path = os.path.join(results_dir, "icl", translation_direction, f'{translation_direction}_{shot}_example_prompt.json')

        print('______________________________________________________________')
        print(f'File path to be loaded: {file_path}')
        print('______________________________________________________________')

        # Load json file as DataFrame
        current_df = pd.DataFrame(load_file(file_path))

        # Median splitting based on source language
        current_df_names, current_dfs = split_by_median(source_language, median_splitting, current_df)

        # Loop of current dfs
        for name, df in zip(current_df_names, current_dfs):

            # Print statement
            print('______________________________________________________________')
            print(name)

            # Extract mt column
            mt_column = df.columns[3]
            print(f'Machine-translated column name: {mt_column}')
            print(f'Source language: {source_language}')
            print(f'Target language: {target_language}')
            print('______________________________________________________________')


            # Predict COMET scores
            model_output = run_comet(source_language, target_language, mt_column, df, model)

            # Print the system score
            print(f'System score for {translation_direction}_{shot} in dataset {name}: {model_output.system_score}')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


______________________________________________________________
______________________________________________________________
DE_to_EN
______________________________________________________________
______________________________________________________________
______________________________________________________________
File path to be loaded: /content/drive/MyDrive/historical_mt/results/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/icl/DE_to_EN/DE_to_EN_000_example_prompt.json
______________________________________________________________
Shape of current_df_short: (507, 5)
Shape of current_df_long: (493, 5)
Median word count in Early Modern Bohemian German: 74.0
______________________________________________________________
current_df_short
Machine-translated column name: DE_to_EN_000_example_prompt
Source language: Early Modern Bohemian German
Target language: English
______________________________________________________________


Predicting DataLoader 0: 100%|██████████| 64/64 [00:07<00:00,  8.25it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


System score for DE_to_EN_000 in dataset current_df_short: 0.5514847951880574
______________________________________________________________
current_df_long
Machine-translated column name: DE_to_EN_000_example_prompt
Source language: Early Modern Bohemian German
Target language: English
______________________________________________________________


Predicting DataLoader 0: 100%|██████████| 62/62 [00:17<00:00,  3.52it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

System score for EN_to_DE_128 in dataset current_df_long: 0.5520580585910241
______________________________________________________________
File path to be loaded: /content/drive/MyDrive/historical_mt/results/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/EN_to_DE/EN_to_DE_finetuning.json
______________________________________________________________
Shape of current_df_short: (501, 5)
Shape of current_df_long: (499, 5)
Median word count in English: 80.0
______________________________________________________________
current_df_short
Machine-translated column name: EN_to_DE_finetuning
Source language: English
Target language: Early Modern Bohemian German
______________________________________________________________


Predicting DataLoader 0: 100%|██████████| 63/63 [00:08<00:00,  7.87it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


System score for EN_to_DE_finetuning in dataset current_df_short: 0.5471379339040634
______________________________________________________________
current_df_long
Machine-translated column name: EN_to_DE_finetuning
Source language: English
Target language: Early Modern Bohemian German
______________________________________________________________


Predicting DataLoader 0: 100%|██████████| 63/63 [00:20<00:00,  3.04it/s]


System score for EN_to_DE_finetuning in dataset current_df_long: 0.5985305572977525
